# Script for Generating Training Data from Frames using PIPs
https://github.com/aharley/pips

## Initial set up
Connection to MyDrive, requirements, imports, downloading models weights

In [1]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# installing requirements
!pip install imageio==2.19.3
!pip install opencv-python==4.6.0.66
!pip install einops==0.4.1
!pip install scikit-learn==1.1.1
!pip install matplotlib==3.5.1
!pip install protobuf==3.20.0
!pip install fire==0.4.0

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/drive_folder/pips')
import time
import numpy as np
import io
import os
from PIL import Image
import cv2
import saverloader
from nets.pips import Pips
import utils.improc
import random
import glob
from utils.basic import print_, print_stats
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gc
import imageio.v2 as imageio
from csv import writer
# imports pytorch
import torch

In [ ]:
# downloading their reference model
!bash /content/drive/MyDrive/drive_folder/pips/get_reference_model.sh

## Loading zip folder with data frames
used to compute particle flow

In [ ]:
DATA_DIR = "frames" # specify the name of the data folder located in drive
!unzip -d "$DATA_DIR"/ /content/drive/MyDrive/"$DATA_DIR".zip # unziping frames

In [ ]:
# List of videos to go through
videos_list = [f for f in os.listdir(DATA_DIR+'/content/frames/') if not f.startswith('.')]
print(videos_list)

## Generating training data

## Helper functions

In [ ]:
def img_to_array(tensor_img):
  """
  Change tensor to numpy array for storing of images
  - resizing to 640x360 (to match coords resolution)
  - normalising pixel value to range 0-1 (required for input to FCN ResNet-50)
  """
  x = tensor_img.permute(1, 2, 0).numpy()
  x = cv2.resize(x, (640, 360))
  x = x.astype(float) / 255
  return x

def save_data(sample_id, frame0, frame3, frame7, trajs, vis):
  """
  Saving
  - coordinates to COORDS_DIR as .npy files
  - visibility of coordinates to VIS_DIR as .npy files
  - 0th frames to FRAME0_DIR directory as .npy files
  - 7th frames to FRAME7_DIR directory as .npy files
  
  - Coordinates, visibility and frames are mapped with sample_id, all sample_ids are stored in sample_ids.txt
  """
  img0 = img_to_array(frame0) # transform 0th frame to array
  img3 = img_to_array(frame3) # transform 2nd frame to array
  img7 = img_to_array(frame7) # tranform last frame (8th) to array
  coords = trajs.cpu().numpy()
  vis = vis.cpu().numpy()

  # Store coords, frame0, frame1
  np.save(COORDS_DIR+sample_id, coords)
  np.save(VIS_DIR+sample_id, vis)
  np.save(FRAME0_DIR+sample_id, img0)
  np.save(FRAME3_DIR+sample_id, img3)
  np.save(FRAME7_DIR+sample_id, img7)

  # Add sample_id to log list
  LOG_LIST.append(sample_id)
  save_log(LOG_LIST)

def save_log(log_list):
  with open(LOG_FILE, "w") as outfile:
    outfile.write("\n".join(log_list))

## PIPs implementation

In [ ]:
def create_model(model_weights):
  model = Pips(stride=4).cuda()
  parameters = list(model.parameters())
  _ = saverloader.load(model_weights, model)
  global_step = 0
  model.eval()
  return model

def run_model(model, rgbs, N, split):
    rgbs = rgbs.cuda().float() # B, S, C, H, W

    B, S, C, H, W = rgbs.shape
    rgbs_ = rgbs.reshape(B*S, C, H, W)
    H_, W_ = 360, 640
    rgbs_ = F.interpolate(rgbs_, (H_, W_), mode='bilinear')
    H, W = H_, W_
    rgbs = rgbs_.reshape(B, S, C, H, W)

    # Pick N points to track; we'll use a uniform grid
    N_ = np.sqrt(N).round().astype(np.int32)
    grid_y, grid_x = utils.basic.meshgrid2d(B, N_, N_, stack=False, norm=False, device='cuda')
    grid_y = 8 + grid_y.reshape(B, -1)/float(N_-1) * (H-16)
    grid_x = 8 + grid_x.reshape(B, -1)/float(N_-1) * (W-16)
    xy = torch.stack([grid_x, grid_y], dim=-1) # B, N_*N_, 2
    _, S, C, H, W = rgbs.shape

    print_stats('rgbs', rgbs)

    # Splitting grid of points into tensors of size 1500 along dim=1
    if split:
       xy_split = torch.split(xy, 1500, dim=1)
       preds_split = []
       vis_split = []

      # For each splitted point compute trajs
       for i in range(len(xy_split)):
         preds, preds_anim, vis, stats = model(xy_split[i], rgbs, iters=6)
         preds_split.append(preds[-1])
         vis_split.append(vis[-1])

       # Put trajs back together
       trajs_e = torch.cat(preds_split, 2)
       vis_e = torch.cat(vis_split, 1).unsqueeze(0)
       

    else:
      preds, preds_anim, vis, stats = model(xy, rgbs, iters=6)
      # preds is a list of torch tensors 
      trajs_e = preds[-1] # tensor of shape (frames, tracking points, 2)
      vis_e = vis

    print_stats('vis_e', vis_e)
    return trajs_e[:,-1,:,:], vis_e[:,-1,:]

def generate_training_data(pips_model, video_name, n, split=True):
    model = pips_model
    
    # Choose hyps
    B = 1
    S = 8
    N = n**2 # number of points to track

    # Get frames of a file
    filenames = glob.glob(DATA_DIR+'/content/frames/'+video_name+'/frames/*.jpg')
    filenames = sorted(filenames)

    # Remove every second frame from a list
    n = 2
    del filenames[n - 1::n]

    max_iters = (len(filenames)//S)-100 # run each unique subsequence
    global_step = 100

    # Run model each of 8 frames, from frame 100 to frame end-100 (to avoid titles in the video)
    while global_step < max_iters:
        global_step += 1
        
        try:
            rgbs = []
            sample_id = video_name[-11:]+"_"+str((global_step-1)*S)
            print("sample {}: step {}/{}".format(sample_id, global_step, max_iters))

            # Skip generating this sample if it is already in the log list
            if sample_id in LOG_LIST:
                  print(sample_id+" already in log list")
                  continue

            for s in range(S):
                frame_num = (global_step-1)*S+s
                fn = filenames[frame_num]
                if s==0:
                    print('start frame', fn)
                im = imageio.imread(fn)
                im = im.astype(np.uint8)
                im = torch.from_numpy(im).permute(2,0,1)
                rgbs.append(im)
            rgbs = torch.stack(rgbs, dim=0).unsqueeze(0) # 1, S, C, H, W

            with torch.no_grad():
                trajs_e, vis_e = run_model(model, rgbs, N, split)

            # Store img0, img3 and img7 (first, middle and last frame) and trajs_e, vis_e
            save_data(sample_id, rgbs[0][0], rgbs[0][3], rgbs[0][-1], trajs_e, vis_e)

        except FileNotFoundError as e:
            print('error', e)

In [ ]:
model = create_model('reference_model')

## Creating/Loading folder to store data in

In [ ]:
# Run this cell if you want to create a new dataset,
# will create a folder to store training data in
!rm -r training_data
!mkdir training_data
!mkdir training_data/frame0/
!mkdir training_data/frame3/
!mkdir training_data/frame7/
!mkdir training_data/coords/
!mkdir training_data/vis/
!touch training_data/sample_ids.txt
PATH = "/content/training_data"

rm: cannot remove 'training_data': No such file or directory


In [ ]:
# Run this cell if we want to use an old training data folder
# specify the name of the data folder located in drive
TRAIN_DATA_DIR = "full_dataset_2023-02-18"
!unzip -d "$TRAIN_DATA_DIR"/ /content/drive/MyDrive/"$TRAIN_DATA_DIR".zip

# Set path to that folder
PATH = "/content/"+TRAIN_DATA_DIR

In [6]:
TRANING_DATA_DIR = PATH+"/"
COORDS_DIR = PATH+"/coords/"
VIS_DIR = PATH+"/vis/"
FRAME0_DIR= PATH+"/frame0/"
FRAME3_DIR= PATH+"/frame3/"
FRAME7_DIR= PATH+"/frame7/"
LOG_FILE = PATH+"/sample_ids.txt"
LOG_LIST = open(LOG_FILE).read().splitlines()

### Deleting corrupted files

In [ ]:
import os
import fnmatch
corrupted = ["-CR4xjdQbkc", "MGYF1aDwUKg", "vB8XTJfV4rY", "KjMxdYJOwqI", "asL3ZyuNeB0", "KMO1BluPtU4", "sm6-dbG5Rho"]
folders = ['/frame0/', '/frame3/', '/frame7/', '/coords/', '/vis/']
for c in corrupted:
  print(c)
  for folder in folders:
    for file in os.listdir(PATH+folder):
      if fnmatch.fnmatch(file, c+'*.npy'):
        os.remove(PATH+folder+'/'+file)

## Generating data

In [ ]:
videos_part1 = ['Tai chiKunlun Si Xiang Quan Demonstrated by Huang Shuanqing--CR4xjdQbkc', #corrupted
                'Tai chi陈式太极拳五十六式-vB8XTJfV4rY', # corrupted
                'Tai chiTrần thức giản hóa do Vs - Thiều Ngọc Sơn diễn luyện.-MGYF1aDwUKg', #corrupted
                'Tai chiYang Taijiquan 16 vorm-KjMxdYJOwqI', #corrupted
                ]

for v in videos_part1:
  print("Now generating training data for {}.".format(v))
  generate_training_data(model, v, n=64)

In [ ]:
videos_part2 = ['Tai chiTai Chi 32 Yang Stil Schwertform  _ 32 Step Yang Style Tai Chi Sword Routine-asL3ZyuNeB0', #corrupted,
                'Tai chi吳式太極快架-KMO1BluPtU4', # corrupted
                'Tai chiTai Chi sword form-sm6-dbG5Rho' #corrupted
                ]

for v in videos_part2:
  print("Now generating training data for {}.".format(v))
  generate_training_data(model, v, n=64)

In [ ]:
videos_part3 = ['Tai chizhaobao Taichi 24 刘钢 示范 武当赵堡和式太极拳二十四式简化太极拳-FmJ33H476pI',
                'Tai chi56式夕陽美功夫扇-uOw-z7CR7x8',
                'Tai chiTai Chi for Osteoporosis Front Demo-AUzpWyGi8Gw',
                'Tai chichen 56-l0aKiGrTUGw',
                'Tai chi陳氏太極拳：老架一路(上)王西安.flv-CA9-8CydW4A',
                'Tai chiTaijiquan Tai Chi Pai Lin part 2-6f6nS3PCeQM'
                ]

for v in videos_part3:
  print("Now generating training data for {}.".format(v))
  generate_training_data(model, v, n=64)

In [ ]:
videos_part4 = [
                'Tai chiSun style - short form (30 forms)-FzJJ_bWLzcA',
                "Tai chiTai Chi Chuan -Yang Style - Grand Master William C.C. Chen's 60 Revolving Movements - 1st Section-DXO_BbscaFg",
                'Tai chiGuang Ping Yang Tai Ji Quan (Tai Chi Chuan)-VidZQ6yA7I4']

for v in videos_part4:
  print("Now generating training data for {}.".format(v))
  generate_training_data(model, v, n=64)

In [ ]:
videos_part5 = [
                'Tai chiTai Chi Chuan-vFDuL2-zhvQ',
                'Tai chi陳式太極拳56式 - 游振芳 吳志芬-ypIdWCYjHuo',
                'Tai chiLiu He Ba Fa performed by Paul Dillon-BKLJ4mRzYuE',
                'Tai chiInstitut Français de Taï Ji Zhang Dongwu-jFuiqvVaESE']

for v in videos_part5:
  print("Now generating training data for {}.".format(v))
  generate_training_data(model, v, n=64)

## Zip data and store in Drive

### Check data was generated correctly

In [ ]:
from datetime import date
TODAY = date.today()
FILENAME = "full_dataset"

# zip training data
!cd "$PATH" && zip -r /content/"$FILENAME"_"$TODAY".zip .
# cp NumPy zip file into drive
!cp "$FILENAME"_"$TODAY".zip /content/drive/MyDrive/

In [ ]:
import os
import fnmatch

x = [f for f in os.listdir(PATH+'/coords/') if not f.startswith('.')] 
for name in x:
  if fnmatch.fnmatch(name, 'ypIdWCYjHuo'+'*.npy'):
     print(name, np.load(PATH+'/coords/'+name, encoding='bytes').shape)
     coords = np.load(PATH+'/coords/'+name, encoding='bytes')
     coords = coords.squeeze()
     plt.scatter(coords[:, 0], coords[:, 1], s=0.5, marker='.', cmap=plt.cm.coolwarm)
     plt.ylim(max(plt.ylim()), min(plt.ylim()))
     plt.pause(0.001)  # pause a bit so that plots are updated

## Split dataset

In [15]:
!rm -r train_data
!mkdir train_data
!mkdir train_data/frame0/
!mkdir train_data/frame3/
!mkdir train_data/frame7/
!mkdir train_data/coords/
!mkdir train_data/vis/
!touch train_data/sample_ids.txt
PATH_TRAIN = "/content/train_data"

!rm -r validation_data
!mkdir validation_data
!mkdir validation_data/frame0/
!mkdir validation_data/frame3/
!mkdir validation_data/frame7/
!mkdir validation_data/coords/
!mkdir validation_data/vis/
!touch validation_data/sample_ids.txt
PATH_VALID = "/content/validation_data"

!rm -r test_data
!mkdir test_data
!mkdir test_data/frame0/
!mkdir test_data/frame3/
!mkdir test_data/frame7/
!mkdir test_data/coords/
!mkdir test_data/vis/
!touch test_data/sample_ids.txt
PATH_TEST = "/content/test_data"

LOG_FILE = PATH+"/sample_ids.txt"
LOG_LIST = open(LOG_FILE).read().splitlines()

In [8]:
def save_log_to_file(log_list, file):
  with open(file, "w") as outfile:
    outfile.write("\n".join(log_list))

In [13]:
# 80/10/10 train/valid/test split
np.random.seed(42)
np.random.shuffle(LOG_LIST)

train_list = LOG_LIST[:int(len(LOG_LIST)*0.8)]
valid_list = LOG_LIST[len(train_list):len(train_list)+int(len(LOG_LIST)*0.1)]
test_list = LOG_LIST[len(train_list)+len(valid_list):]

print(len(train_list))

3656


In [16]:
import shutil
def save_split_dataset(path_from, path_to, sample_id_list):
  log_file = path_to+"/sample_ids.txt"
  log_list = open(log_file).read().splitlines()

  for sample_id in sample_id_list:
    shutil.move(path_from+"/frame0/"+sample_id+".npy", path_to+"/frame0/")
    shutil.move(path_from+"/frame3/"+sample_id+".npy", path_to+"/frame3/")
    shutil.move(path_from+"/frame7/"+sample_id+".npy", path_to+"/frame7/")
    shutil.move(path_from+"/coords/"+sample_id+".npy", path_to+"/coords/")
    shutil.move(path_from+"/vis/"+sample_id+".npy", path_to+"/vis/")
    log_list.append(sample_id)
  save_log_to_file(log_list, log_file)

  print(len(log_list))


In [18]:
save_split_dataset(PATH, PATH_TRAIN, train_list)
save_split_dataset(PATH, PATH_VALID, valid_list)
save_split_dataset(PATH, PATH_TEST, test_list)

457
458


In [ ]:
from datetime import date
TODAY = date.today()
FILENAME_TRAIN = "full_dataset_train"
FILENAME_VALID = "full_dataset_valid"
FILENAME_TEST = "full_dataset_test"

# zip train data
!cd "$PATH_TRAIN" && zip -r /content/"$FILENAME_TRAIN"_"$TODAY".zip .
# cp NumPy zip file into drive
!cp "$FILENAME_TRAIN"_"$TODAY".zip /content/drive/MyDrive/

# zip validation data
!cd "$PATH_VALID" && zip -r /content/"$FILENAME_VALID"_"$TODAY".zip .
# cp NumPy zip file into drive
!cp "$FILENAME_VALID"_"$TODAY".zip /content/drive/MyDrive/

# zip test data
!cd "$PATH_TEST" && zip -r /content/"$FILENAME_TEST"_"$TODAY".zip .
# cp NumPy zip file into drive
!cp "$FILENAME_TEST"_"$TODAY".zip /content/drive/MyDrive/